# 1.Data Collection

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from nltk.stem import *
import time
import nltk
from nltk.corpus import stopwords
from collections import Counter
import random
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re
from forex_python.converter import CurrencyRates
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
url = 'https://www.findamasters.com/masters-degrees/msc-degrees/'
result = requests.get(url)
soup = BeautifulSoup(result.text)
lst_links = soup.find_all('a', {'class': 'courseLink'})

In [3]:
def extract_masters(this_url):
  result_url = requests.get(this_url)
  result_soup = BeautifulSoup(result_url.text)
  result_links = result_soup.find_all('a', {'class': 'courseLink'})
  result_list = []
  for item in result_links:
    result_list.append((item['href'], item.text))

  return result_list

In this first part, we have created a txt file containing the list of master's degree courses.

In [ ]:
'''
num_pages = 400
pref = 'https://www.findamasters.com'
test_lst_all = []

for i in range(1, num_pages + 1):
    test_lst = extract_masters(pref + '/masters-degrees/msc-degrees/?PG=' + str(i))
    test_lst_all.extend(test_lst)
with open('masters_urls.txt', 'w') as f:
    for item in test_lst_all:
        f.write(item[0] + '\n')

'''

Save HTML file
Then, we wrote a function to save the data extracted from the HTML pages, which we stored in a folder named 'pages_html'

In [ ]:
# Impostazioni per l'User-Agent per simulare un browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
folder_name = "html-pages"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Initialize an empty list to store extracted data
data_list = []

# Your loop for creating folders
for i in range(1, 401):
    name = 'HTML page ' + str(i)
    path_folder = os.path.join(folder_name, name)
    if not os.path.exists(path_folder):
        os.makedirs(path_folder)



In [ ]:
from extract_masters import extract_msc_page

In [ ]:
# Apre il file contenente gli URL
with open('masters_urls.txt', 'r') as file:
    for index, url in enumerate(file):
        url = url.strip()
        page = (index // 15) + 1
        try:
            full_url = "https://www.findamasters.com" + url.strip()
            time.sleep(1 + random.uniform(0, 4))
            response = requests.get(full_url, headers=headers)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                file_path = os.path.join(f"{folder_name}{os.path.sep}HTML page {page}", f"course {index+1}.html")
                with open(file_path, "w", encoding="utf-8") as html_file:
                    html_file.write(str(soup))

                # Extract data and append to the list
                data_list.append(extract_msc_page(full_url))

            else:
                print(f"Error retrieving the course page:{full_url}")

        except Exception as e:
            print(f"Error during retrieval and saving of the page {full_url}: {e}")



In [ ]:
# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data_list)

# Save the DataFrame to a CSV file
df.to_csv("masters_dataset.csv", index=False)

In [4]:
df = pd.read_csv(r"C:\Users\rebbi\Desktop\Magistrale\ADM\Adm_hw3\HW3_group1\masters_dataset.csv")
df.head(3)

,url,courseName,universityName,facultyName,fullTime,description,startDate,fees,modality,duration,city,country,administator
0,https://www.findamasters.com/masters-degrees/c...,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,True,3D visualisation and animation play a role in ...,September,FeesPlease see the university website for furt...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus
1,https://www.findamasters.com/masters-degrees/c...,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,True,Businesses and governments rely on sound finan...,September,"FeesUK: £18,000 (Total)International: £34,750 ...",MSc,1 year full time,Leeds,United Kingdom,On Campus
2,https://www.findamasters.com/masters-degrees/c...,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,True,"Our Accounting, Accountability & Financial Man...",September,FeesPlease see the university website for furt...,MSc,1 year FT,London,United Kingdom,On Campus


Create tsv files\
To work easily with the dataset, we saved all the data in a CSV file, and subsequently, each row, corresponding to each course, was saved in a TSV (Tab-Separated Values) file

In [7]:
# Save each row as a separate TSV file
output_folder = 'course_data_tsv'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for index, row in df.iterrows():
    course_name = f"course_{index + 1}.tsv"
    output_path = os.path.join(output_folder, course_name)
    row.to_csv(output_path, sep='\t', index=False)


# 2.Search Engine

Preprocessing

In [6]:
# Download NLTK resources if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define or import your list of stopwords and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# Initialize stemmer
stemmer = PorterStemmer()

# Function to preprocess text
def preprocess_text(text):
    if pd.isnull(text):
        return ""  # Return an empty string for missing values
    tokens = word_tokenize(str(text).lower())  # Ensure text is converted to string before processing
    processed_tokens = [word for word in tokens if word.isalnum() and word not in stop_words and word not in punctuation]
    return ' '.join(processed_tokens)

# Assuming df is your DataFrame and 'description' is the column to preprocess
df['preprocessed_desc'] = df['description'].apply(preprocess_text)

# Function to perform stemming on a sentence
def stem_sentence(sentence):
    return [stemmer.stem(word) for word in word_tokenize(sentence)]

# Apply stemming to 'preprocessed_desc' column
df['descr_stem'] = df['preprocessed_desc'].apply(stem_sentence)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rebbi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rebbi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Display or use the DataFrame with the 'descr_stem' column containing lists of stemmed words
print(df['descr_stem'].head())

0    [3d, visualis, anim, play, role, mani, area, p...
1    [busi, govern, reli, sound, financi, knowledg,...
2    [account, account, financi, manag, msc, cours,...
3    [embark, profession, account, career, academ, ...
4    [join, us, onlin, session, prospect, student, ...
Name: descr_stem, dtype: object


Preprocessing the fees column\
For the 'fees' column, we decided to convert the numerical values into USD.

In [9]:
# Function to extract numerical fee values
def extract_numerical_fee(text):
    numerical_values = re.findall(r'\d+\.?\d*', str(text))  # Ensure text is converted to string
    return max(map(float, numerical_values)) if numerical_values else None  # Return the maximum value

# Clean textual fee data and extract numerical values
df['numeric_fee'] = df['fees'].apply(lambda x: extract_numerical_fee(x) if isinstance(x, str) else None)

# Ensure the 'currency_code' column exists and contains valid currency codes
df['currency_code'] = 'USD'  # Replace 'USD' with the correct currency code column

def convert_to_common_currency(row):
    if pd.notnull(row['numeric_fee']) and pd.notnull(row['currency_code']):
        c = CurrencyRates()
        conversion_rate = c.get_rate(row['currency_code'], 'USD')
        return row['numeric_fee'] * conversion_rate
    else:
        return None

df['fees_usd'] = df.apply(convert_to_common_currency, axis=1)



In [10]:
print(df['fees_usd'].head() )

0      NaN
1    750.0
2      NaN
3      NaN
4      NaN
Name: fees_usd, dtype: float64


Many of the values are null due to them being missing on the main pages.

Conjunctive query\
For the first version of the search engine, we narrowed our interest to the description of each course. It means that you will evaluate queries only concerning the course's description.

First, we created a vocabulary with all the words within the 'description' column. Then, we removed stopwords, punctuations, and performed stemming.
As a final step, we created the inverse index and then displayed both the vocabulary and the inverse index list.

In [24]:
df['text'] = df['descr_stem'].apply(lambda x: ' '.join(x))

# Create an empty vocabulary and inverted index
vocabulary = {}
inverted_index = defaultdict(list)
term_id = 1  # Start term_id from 1

def tokenize(text):
    if pd.isnull(text):
        return []
    # Split the preprocessed and stemmed text into words
    return text.split()

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    words = tokenize(row['text'])
    
    # Keep track of unique words in each document to avoid duplicate indexing
    unique_words_in_document = set()
    
    # Loop through each word in the text
    for word in words:
        if word not in vocabulary:
            vocabulary[word] = term_id
            term_id += 1
        
        term_id_for_word = vocabulary[word]

        # Ensure the document is added only once for each term
        if index not in inverted_index[term_id_for_word]:
            inverted_index[term_id_for_word].append(index)
            
            # Add the word to the set of unique words in this document
            unique_words_in_document.add(term_id_for_word)



In [29]:
# Existing function to print vocabulary
def print_vocabulary(vocabulary, num_rows):
    count = 0
    for word, term_id in vocabulary.items():
        if count < num_rows:
            print(f"Word: {word}, Term ID: {term_id}")
            count += 1
        else:
            break

print("\nVocabulary")
print_vocabulary(vocabulary, 3)

print("\nInverted Index:")
print_vocabulary(inverted_index,2)


Vocabulary
Word: 3d, Term ID: 1
Word: visualis, Term ID: 2
Word: anim, Term ID: 3

Inverted Index:
Word: 1, Term ID: [0, 372, 487, 488, 783, 1618, 2122, 2500, 2502, 2503, 3520, 3609, 4448]
Word: 2, Term ID: [0, 49, 51, 327, 635, 971, 1175, 1191, 1322, 1323, 1326, 1503, 1521, 1522, 1858, 1987, 2272, 2295, 2297, 2298, 2300, 2304, 2305, 2322, 2328, 2369, 2371, 3384, 3566, 3568, 3595, 3737, 4568, 4570]


Execute the query

In [11]:
def search_documents(query):
    # Preprocess the query similarly to how you processed the descriptions
    query_tokens = word_tokenize(preprocess_text(query))
    query_tokens = [stemmer.stem(word) for word in query_tokens if word not in stop_words and word not in punctuation]

    # Initialize a list to store information for the relevant documents
    relevant_docs_info = []

    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the stemmed tokens from the 'descr_stem' column for the current document
        doc_tokens = set(row['descr_stem'])

        # Check if all query tokens are present in the current document
        if all(token in doc_tokens for token in query_tokens):
            relevant_doc_info = {
                'courseName': row['courseName'],
                'universityName': row['universityName'],
                'description': row['description'],
                'url': row['url']
            }
            relevant_docs_info.append(relevant_doc_info)

    # Create a new DataFrame from the collected information
    relevant_docs_df = pd.DataFrame(relevant_docs_info)

    return relevant_docs_df


We searched among the courses for those that had 'Business' in their description, and these are the first 4 documents that contain it.

In [13]:
# Execute a query
user_query = "Business"
relevant_documents = search_documents(user_query)
relevant_documents.head()



,courseName,universityName,description,url
0,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...
1,"Accounting, Financial Management and Digital B...",University of Reading,Embark on a professional accounting career wit...,https://www.findamasters.com/masters-degrees/c...
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Programme overviewThe Advanced Master in Innov...,https://www.findamasters.com/masters-degrees/c...
3,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Goal of the pro­grammeAre you looking forward ...,https://www.findamasters.com/masters-degrees/c...
4,Applied Statistics (online) MSc,University of Strathclyde,Ouronline MSc in Applied Statisticsis a conver...,https://www.findamasters.com/masters-degrees/c...


For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

- Find all the documents that contain all the words in the query.
- Sort them by their similarity with the query.
- Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use  Python libraries) for maintaining the top-k documents.

To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the description. Let's see how.

In [16]:

# Fit and transform the text using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# Get the feature names (terms) from the vectorizer
terms = tfidf_vectorizer.get_feature_names_out()

# Create an inverted index
inverted_index_tfidf = {}


# Loop through each term's index and documents to build the inverted index
for term_index, term in enumerate(terms):
    inverted_index_tfidf[term] = []
    for document_index in range(len(df)):  # Loop through document indices directly
        tfidf_score = tfidf_matrix[document_index, term_index]
        if tfidf_score > 0:
            # Use the document index as the identifier
            inverted_index_tfidf[term].append((df.index[document_index], tfidf_score))




print the first five element to see if it is working

In [21]:
print(inverted_index_tfidf['aim'][1:5])

[(33, 0.05658503485848769), (63, 0.12466950401056), (65, 0.1045107122490147), (75, 0.10475124441000583)]


In [18]:

query = "aim"
query_tfidf = tfidf_vectorizer.transform([query])  # Transform the query into TF-IDF representation

# Calculate cosine similarity between the query and all documents
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

# Create a DataFrame to store document information and similarity scores
result_df = pd.DataFrame({
    'courseName': row['courseName'],
    'universityName': df['universityName'],
    'description': df['description'],
    'url': df['url'],
    'similarity_score': cosine_similarities
})

# Sort the documents based on similarity scores in descending order
result_df = result_df.sort_values(by='similarity_score', ascending=False)



For this last part, we displayed the documents containing the word 'aim' with a higher cosine similarity.

In [19]:
# Format similarity scores to have five decimal places
result_df['similarity_score'] = result_df['similarity_score'].apply(lambda x: round(x, 5))

# Print the sorted DataFrame
result_df.head()

,courseName,universityName,description,url,similarity_score
2962,Materials Engineering,Cardiff Metropolitan University,The aim of the MSc Entrepreneurship and Innova...,https://www.findamasters.com/masters-degrees/c...,0.26415
4764,Materials Engineering,Cardiff University,Why study this courseThis course aims to equip...,https://www.findamasters.com/masters-degrees/c...,0.26325
508,Materials Engineering,University College London,Register your interest in graduate study at UC...,https://www.findamasters.com/masters-degrees/c...,0.24586
1025,Materials Engineering,Vytautas Magnus University,The aim of the Applied Informatics Master stud...,https://www.findamasters.com/masters-degrees/c...,0.24046
2226,Materials Engineering,Coventry University,The MSc in Cyber Security aims to provide you ...,https://www.findamasters.com/masters-degrees/c...,0.23876


It seems good that the same courses in different universitiess have high similarity.


# 4. Visualising the most relevant MSc degrees
1. Merging query result and coordinates
2. Visualising the query result on a map

In [ ]:
import general_functions
import collecting_coords

## 4.1. Merging query result and coordinates
We have collected the coordinates of the universities mostly by geo-coding (using nominatim). You can look up the explicit code in collecting_coords.py. Since we saved the output to csv-files, we won't execute them again. Next we just get the query, add the coordinates and show the first three examples.

In [ ]:
query_result = general_functions.get_query_result()
query_result = collecting_coords.add_coords(query_result)
query_result[['universityName','lat','lng']].drop_duplicates().head(3)

,universityName,lat,lng
0,Glasgow Caledonian University,55.867466,-4.250245
1,University of Leeds,53.806774,-1.556288
2,King’s College London,51.506000,-0.112158


## 4.2. Visualising the query result on a map
You will see a map created by folium with adjusted zoom-level. The locations indicated are the coordinates of the universities of the courses listed in the query_result. If the query_result contains multiple courses of the same university, the courses are enumerated. You can see the courses together with possible fees by clicking on the marked locations.
Further each location is linked to the other locations of the query result. Moving the cursor over this line yields information about the universities connected by this line and the distance between them.

In [ ]:
import map_visualisation
map_visualisation.show_map(map_visualisation.beautify(query_result))

# 7. Algorithmic Question

1. Implement a code to solve the above mentioned problem.

2. What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.

3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the two differ, which one is right? (why?)

4. What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.

## 7.1 Implementing a Solution

Our solution is based on binary search. Basically our solution works as follows:
1. add up Flexibility (maximal working hours - minimal working hours) and store it in an array (this array is by construction sorted)
2. compute remaining working hours (i.e. hours not already consumed by the minimal working hours)
3. search (by binary search) for the first index (critical day) in the Flexibility array, where Flexibility is greater than or equal to the remaining working hours.
4. Print upper working hours until the critical day, print the hours for the critical day, print the minimal working hours for the remaining days. 

Let us first define an appropriate version of binary search.

In [ ]:
def binary_search_geq(arr, constant, low, high):
    """
    Takes a sorted array (arr) as well as the indexes low and high.
    Assumes low <= high.
    Searches for the first index where the corresponding value is greater than or equal to a constant.
    Returns the found index
    This method is just a properly adjusted version of the binary_search-algorithm.
    """
    # if low <= high:
    mid = (low+high)//2
    # print(mid)
    if arr[mid] >= constant:
        if mid == 0 : # is it the first element? 
            return mid #, arr[mid] - constant # We've found the searched for tuple!
        elif arr[mid - 1] < constant: # is the value at index mid-1 still <= 0 ?
            return mid # , arr[mid] - constant # We've found the searched for tuple!
        else:
            return binary_search_geq(arr, constant, low, mid - 1) # search goes on in the left part of the array.
    else: 
        return binary_search_geq(arr, constant, mid + 1, high) # search goes on in the right part of the array

It follows the code to run the problem.

In [ ]:
# d stands for "total_days
def provide_report():
    d, sumHours = map(int, input().split())
    remHours = sumHours # remaining hours
    total_flexibility = 0
    lower_bounds = []
    upper_bounds = []
    cumFlex = [] # cumulated flexibility


    for _ in range(d):
        lower, upper = map( int, input().split() )
        lower_bounds.append(lower)
        upper_bounds.append(upper)
        remHours -= lower
        total_flexibility += upper-lower
        cumFlex.append(total_flexibility) # this gives a sorted array

    if remHours < 0 or remHours > total_flexibility:
        print('NO')
    else:
        print('YES')
        crit_day = binary_search_geq(cumFlex, remHours, 0 , d-1)
        crit_hours = lower_bounds[crit_day]
        if crit_day > 0:
            crit_hours += remHours - cumFlex[crit_day - 1] # greater than or equal to zero by above binary search.
        else:
            crit_hours += remHours

        working_more_than_minimal = upper_bounds[:crit_day]
        working_more_than_minimal.append(crit_hours)

        # report = working_more_than_minimal + lower_bounds[crit_day+1:]
        print(*working_more_than_minimal, *lower_bounds[crit_day+1:] )


Examples

In [ ]:
provide_report()

YES
10 0


## 7.2 Time Complexity
We consider the function provide_report(). The input needs O(d) time, where d is the total number of days and the print-statement at the end of provide_report() takes also O(d). However this should not be the focus of this consideration. Let's consider the part inbetween, after the last input-pair is given.


The only critical step is calling the function "binary_search_geq()". All the other steps clearly just need constant time. However binary search runs in O(log(d)) by lecture where d is the total number of days. And this is optimal for this searching problem.

## 7.3 chatGPT
Chat-GPT agrees on our analysis of the time complexity.

## 7.4 Optimality
**Question 1: Can we start providing a report before receiving the last input-pair?**

In the worst case the answer is "no". The last input-pair can change our complete solution. Consider for example the input
2 2 

0 1 

0 1 

which needs 1 1 as a solution. And the input

2 1 

0 1 

2 2 

needs 0 2 as a solution.



**Question 2: Can the problem be reduced to a simpler problem?**

Possible, but we haven't found a better approach and also believe there is none.

**Resumé**

Hence we believe in the optimality of our code.